In [37]:
___Author___='LumberJack Jyss'

In [38]:
print('Global Optimized LumberJack Environment Motor\n°OoO_FXCM_Oo0°\nLumberJack Jyss 5779(c)')
print('Version v0.02')

Global Optimized LumberJack Environment Motor
°OoO_FXCM_Oo0°
LumberJack Jyss 5779(c)
Version v0.02


In [39]:
print('Importing librairies...')
import fxcmpy
print('version fxcmpy :',fxcmpy.__version__)
import time
import numpy as np
import pandas as pd
import datetime as dt
import talib
from zigzag import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.style.use('seaborn')
%matplotlib inline
import plotly.plotly as py
import plotly.tools as tls
#import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report
print('Librairies imported')

Importing librairies...
version fxcmpy : 1.2.4
Librairies imported


In [40]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [41]:
TOKEN = 'e053ac1597cef331df9429ac8151100ea9f1c411'
server = 'demo'

# minutes: 'm1' , 'm5' , 'm15' , 'm30' 
# hours: 'H1' , 'H2', 'H3', 'H4', 'H6' 'H8'
# one day: 'D1'
# one week: 'W1'
# one month: 'M1'

period = 'm1'
number = 10000
# Time Windows
# start = dt.datetime(2017, 7, 15)
# stop = dt.datetime(2017, 8, 1)
# con.get_candles('EUR/USD', period='D1',start=start, stop=stop)

In [42]:
def connexion(TOKEN,server):
    tmps1=time.time()
    print('Connecting server...')
    con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error',server= server)
    if con.is_connected():
        print('Connection established for',con.get_account_ids(),' - Mode :',server)
    else:
        print('Not connected')
    tmps2=round(time.time()-tmps1,2)
    print ("Connexion executed in = %f" %tmps2,'secondes')
    return(con)

In [43]:
def scrap_data(con,period,number):
    tmps1=time.time()
    data = con.get_candles('SPX500', period=period,number=number) #start=start,stop=stop)
    df = pd.DataFrame()
    df['Open'] = data['askopen']
    df['High'] = data['askhigh']
    df['Low'] = data['asklow']
    df['Close'] = data['askclose']
    df['Date'] = data.index
    df = df[['Date'] + df.columns[:-1].tolist()]
    #df = data.copy()
    tmps2=round(time.time()-tmps1,2)
    print ("Scrap executed in = %f" %tmps2,'secondes')
    return(con,df)

In [44]:
def deep_learning(df):
    
    tmps1=time.time()
    df.reset_index(inplace=True,drop=True)
    dataset_1D = df.copy()
    dataset_1D = dataset_1D.dropna()
    rsi = talib.RSI(dataset_1D['Close'],timeperiod=2)
    upper, middle, lower =  talib.BBANDS(dataset_1D['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(dataset_1D['Close'],timeperiod=5)
    sma8 = talib.SMA(dataset_1D['Close'],timeperiod=8)
    sma10 = talib.SMA(dataset_1D['Close'],timeperiod=10)
    sma12 = talib.SMA(dataset_1D['Close'],timeperiod=12)
    sma15 = talib.SMA(dataset_1D['Close'],timeperiod=15)
    sma30 = talib.SMA(dataset_1D['Close'],timeperiod=30)
    sma35 = talib.SMA(dataset_1D['Close'],timeperiod=35)
    sma40 = talib.SMA(dataset_1D['Close'],timeperiod=40)
    sma45 = talib.SMA(dataset_1D['Close'],timeperiod=45)
    sma50 = talib.SMA(dataset_1D['Close'],timeperiod=50)

    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50

    X = dataset_1D['Close']
    pivots = peak_valley_pivots(X.values, 0.001, -0.001)
    ts_pivots = pd.Series(X, index=X.index)
    ts_pivots = ts_pivots[pivots != 0]

    peak = []
    valley = []

    for i in range(0,dataset_1D.shape[0]):
        if pivots[i] == 1:
            peak.append(pivots[i])
            valley.append(0)
        elif pivots[i] == -1:
            peak.append(0)
            valley.append(pivots[i])
        else:
            peak.append(0)
            valley.append(0)

    bbdelta = upper - middle
    price_bolup = dataset_1D['Close'] - lower
    price_bolow = dataset_1D['Close'] - upper

    rsi5_list = []
    rsi95_list = []
    for i in range(0,dataset_1D.shape[0]):
        try:
            rsi95_list.append(95 - rsi[i])
            rsi5_list.append(rsi[i] - 5)
        except:
            rsi95_list.append(0)
            rsi5_list.append(0)
    varop_spy = dataset_1D['Open'] - dataset_1D['Close']
    varhl_spy = dataset_1D['High'] - dataset_1D['Low']
    dataset_1D['Varop_Spy'] = varop_spy
    dataset_1D['Varhl_spy'] = varhl_spy
    dataset_1D['RSI'] = rsi
    dataset_1D['95 - RSI'] = np.array(rsi95_list)
    dataset_1D['RSI - 5'] = np.array(rsi5_list)
    dataset_1D['BBD_Delta_Up'] = bbdelta
    dataset_1D['delta5_8'] = delta5_8
    dataset_1D['delta8_10'] = delta8_10
    dataset_1D['delta10_12'] = delta10_12
    dataset_1D['delta12_15'] = delta12_15
    dataset_1D['delta15_30'] = delta15_30
    dataset_1D['delta30_35'] = delta30_35
    dataset_1D['delta35_40'] = delta35_40
    dataset_1D['delta40_45'] = delta40_45
    dataset_1D['delta45_50'] = delta45_50
    dataset_1D['Peaks'] = abs(np.array(peak))
    dataset_1D['Valley'] = abs(np.array(valley))
    tsf = talib.TSF(dataset_1D['Close'],timeperiod=14)
    delta_tsf = dataset_1D['Close'] - tsf
    dataset_1D['delta_tsf'] = tsf
    forosc = []
    forosc.append(0)
    for i in range(1,dataset_1D.shape[0]):
        try:
            forosc.append((dataset_1D.iloc[i,4] - tsf[i-1])*100/dataset_1D.iloc[i,4])
        except:
            forosc.append(0)
    dataset_1D['Forcast Oscillator'] = (forosc)
    target_up = []
    target_down = []

    for i in range(0,dataset_1D.shape[0]-5):

        if (dataset_1D.iloc[i+1,5] * dataset_1D.iloc[i,5]) < 0 :
            if (dataset_1D.iloc[i+1,4] - dataset_1D.iloc[i,4]) > 0.2 : 
                target_up.append(1)
                target_down.append(0)
            elif (dataset_1D.iloc[i+1,4] - dataset_1D.iloc[i,4]) < -0.1 : 
                target_up.append(0)
                target_down.append(1)
            else:
                target_up.append(0)
                target_down.append(0)
        else :
            target_up.append(0)
            target_down.append(0) 

    target_up.append(0)
    target_down.append(0)
    target_up.append(0)
    target_down.append(0)
    target_up.append(0)
    target_down.append(0)
    target_up.append(0)
    target_down.append(0)
    target_up.append(0)
    target_down.append(0)
    dataset_1D['target_up'] = target_up  # target_up # abs(np.array(valley))#target_up
    dataset_1D['target_down'] = target_down # target_down # abs(np.array(peak))#target_down
    dataset_1D['target_value'] = dataset_1D['Close']
    dataset_1D = dataset_1D.drop(['Open','High','Low','Close'],axis=1)
    df_1D = dataset_1D.copy()
    df_1D = df_1D.iloc[:,1:]
    df_1D['Date'] = df['Date']
    df_1D = df_1D.dropna()
    df_1D = df_1D[['Date'] + df_1D.columns[:-1].tolist()]
    df_1D.reset_index(inplace=True,drop=True)

    print('GOLEM begins Computing...')
    


    X = df_1D.iloc[:,1:-3]
    y_up = df_1D.iloc[:,-3].values
    y_down = df_1D.iloc[:,-2].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:-1,:]
    Xtest = X[-1:,:]
    ytrain_up = y_up[:-1,:]
    ytest_up = y_up[-1:,:]
    ytrain_down = y_down[:-1,:]
    ytest_down = y_down[-1:,:]

    seed = 770
    np.random.seed(seed)

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    model_up = Sequential()
    # Add an input layer 
    model_up.add(Dense(23, activation='relu'))
    # Add one hidden layer 
    model_up.add(Dense(50, activation='relu'))
    # Add an output layer 
    model_up.add(Dense(1, activation='sigmoid'))

    model_down = Sequential()
    # Add an input layer 
    model_down.add(Dense(23, activation='relu'))
    # Add one hidden layer 
    model_down.add(Dense(50, activation='relu'))
    # Add an output layer 
    model_down.add(Dense(1, activation='sigmoid'))

    print('Processing move_up')
    model_up.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy','mse'])

    history_up = model_up.fit(Xtrain, ytrain_up,epochs=50, batch_size=32, verbose=0)
  
    print('Processing move_down')
    model_down.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy','mse'])

    history_down = model_down.fit(Xtrain, ytrain_down,epochs=50, batch_size=32, verbose=0)

    print('Computing done')
    

    train_acc_up = model_up.evaluate(Xtrain, ytrain_up,verbose=0)
    train_acc_down = model_down.evaluate(Xtrain, ytrain_down,verbose=0)

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    score_up = model_up.evaluate(Xtest, ytest_up,verbose=1)
    score_down = model_down.evaluate(Xtest, ytest_down,verbose=1)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    accuracy_up = accuracy_score(ytest_up, yhat_up)
    accuracy_down = accuracy_score(ytest_down, yhat_down)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    recall_up = recall_score(ytest_up, yhat_up) 
    recall_down = recall_score(ytest_down, yhat_down) 

    resultats = pd.DataFrame()
    resultats['Date'] = df_1D.iloc[-1:,0]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df_1D['target_value']

    tmps2=round(time.time()-tmps1,2)
    print ("DL executed in = %f" %tmps2,'secondes')
    return(resultats)

In [45]:
def go_long(con,resultats,pos_long,pos_short,order_id_long,order_id_short,a):
    
    actual = resultats.iloc[0,5]
    limit = actual + 0.5
    order_id_long = a
    order = con.create_market_buy_order('SPX500', 30)
    print('Go Long')
    pos_long = 1
    return(order_id_long,order_id_short,pos_long,pos_short)

In [46]:
def close_long(con,pos_long,pos_short,order_id_long,order_id_short):
    #close_trade(order_id_long)
    order = con.close_all_for_symbol('SPX500')
    print('Close Long')
    pos_long == 0
    return(pos_long,pos_short)

In [47]:
def go_short(con,resultats,pos_long,pos_short,order_id_long,order_id_short,a):
    actual = resultats.iloc[0,5]
    limit = actual - 0.5
    order_id_short = a
    order = con.create_market_sell_order('SPX500', 30)
    print('Go Short')
    pos_short = 1
    return(order_id_long,order_id_short,pos_long,pos_short)

In [48]:
def close_short(con,pos_long,pos_short,order_id_long,order_id_short):
    #close_trade(order_id_short)
    order = con.close_all_for_symbol('SPX500')
    pos_short == 0
    print('Close Short')
    return(pos_long,pos_short,order_id_long,order_id_short)

In [49]:
def test_pos(con,resultats,pos_long,pos_short,order_id_long,order_id_short,a):
    
    if resultats.iloc[0,1] == 1 and resultats.iloc[0,2] > 75:
        if pos_long == 0 and pos_short == 0:
            order_id_long,order_id_short,pos_long,pos_short = go_long(con,resultats,pos_long,pos_short,order_id_long,order_id_short,a)
        elif  pos_long == 0 and pos_short == 1:
            close_short(con,pos_long,pos_short,order_id_long,order_id_short)

    elif resultats.iloc[0,3] == 1 and resultats.iloc[0,4] > 85:
        if pos_long == 0 and pos_short == 0:
            order_id_long,order_id_short,pos_long,pos_short = go_short(con,resultats,pos_long,pos_short,order_id_long,order_id_short,a)
        elif  pos_long == 1 and pos_short == 0:
            close_long(con,pos_long,pos_short,order_id_long,order_id_short)
            
    return(pos_long,pos_short,order_id_long,order_id_short)

In [50]:
con = connexion(TOKEN,server)
pos_long = 0
pos_short = 0
order_id_long = []
order_id_short = []

Connecting server...


W0614 12:28:43.980911 4730869184 fxcmpy.py:222] Default account set to 1147539, to change use set_default_account().


Connection established for [1147539]  - Mode : demo
Connexion executed in = 8.340000 secondes


In [51]:
for a in range(0,10000) : 
    con,df = scrap_data(con,period,number)
    resultats = deep_learning(df)
    print('Resultats :',resultats,'\n')
    pos_long,pos_short,order_id_long,order_id_short = test_pos(con,resultats,pos_long,pos_short,order_id_long,order_id_short,a)
    

Scrap executed in = 2.350000 secondes
GOLEM begins Computing...
Processing move_up
Processing move_down
Computing done
1/1 [==============================] - 0s 2ms/step
DL executed in = 101.870000 secondes
Resultats :                     Date  Move Up  Confiance up  Move Down  Confiance Down  \
9950 2019-06-14 10:27:00        0      0.000089          1       88.321098   

       Actual  
9950  2881.22   



W0614 12:30:34.508026 123145622745088 fxcmpy.py:2681] Got a insert event for open positions: {'t': 1, 'ratePrecision': 2, 'tradeId': '149765509', 'accountName': '01147539', 'accountId': '1147539', 'roll': 0, 'com': 0, 'open': 2881.16, 'valueDate': '', 'grossPL': -7.9868, 'close': 2881.46, 'visiblePL': -3, 'isDisabled': False, 'currency': 'SPX500', 'isBuy': False, 'amountK': 0.03, 'currencyPoint': 0.00266, 'time': '06142019103034', 'usedMargin': 2100, 'stop': 0, 'stopMove': 0, 'limit': 0, 'action': 'I'}.
W0614 12:30:34.510171 123145612234752 fxcmpy.py:2681] Got a insert event for open positions: {'t': 1, 'ratePrecision': 2, 'tradeId': '149765509', 'accountName': '01147539', 'accountId': '1147539', 'roll': 0, 'com': 0, 'open': 2881.16, 'valueDate': '', 'grossPL': -7.9868, 'close': 2881.46, 'visiblePL': -3, 'isDisabled': False, 'currency': 'SPX500', 'isBuy': False, 'amountK': 0.03, 'currencyPoint': 0.00266, 'time': '06142019103034', 'usedMargin': 2100, 'stop': 0, 'stopMove': 0, 'limit': 0

Go Short
Scrap executed in = 1.520000 secondes
GOLEM begins Computing...
Processing move_up
Processing move_down
Computing done
1/1 [==============================] - 0s 3ms/step
DL executed in = 103.800000 secondes
Resultats :                     Date  Move Up  Confiance up  Move Down  Confiance Down  \
9950 2019-06-14 10:29:00        0           0.0          1       87.884781   

      Actual  
9950  2881.6   

Scrap executed in = 1.330000 secondes
GOLEM begins Computing...
Processing move_up
Processing move_down
Computing done
1/1 [==============================] - 0s 4ms/step
DL executed in = 106.340000 secondes
Resultats :                     Date  Move Up  Confiance up  Move Down  Confiance Down  \
9950 2019-06-14 10:31:00        0           0.0          1       88.272247   

       Actual  
9950  2881.72   

Scrap executed in = 2.100000 secondes
GOLEM begins Computing...
Processing move_up
Processing move_down
Computing done
1/1 [==============================] - 0s 2ms/step
DL 

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
pos_short = 